In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [81]:
#create dataframe from dictionary

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [82]:
#split from_to in different column with proper case
def convertToProperCase(name):
    return name[0].upper()  + name[1:len(name)].lower()


fromsTos = list(df.loc[:,'From_To'])
df['from'] = df['From_To'].apply(lambda x: convertToProperCase(x.split('_')[0]))
df['to'] = df['From_To'].apply(lambda x: convertToProperCase(x.split('_')[1]))

df

,From_To,FlightNumber,RecentDelays,Airline,from,to
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!),London,Paris
1,MAdrid_miLAN,NaN,[],<Air France> (12),Madrid,Milan
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,Budapest_PaRis,NaN,[13],12. Air France,Budapest,Paris
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


In [83]:
# fill nan value as previous value + 10 and convert to integer

flightNumbers = list(df['FlightNumber'])
 
df['FlightNumber'] = [flightNumbers[i-1] + 10 if np.isnan(x) else x for i,x in enumerate(flightNumbers)]

df['FlightNumber'] = df['FlightNumber'].apply(np.int32)

df

,From_To,FlightNumber,RecentDelays,Airline,from,to
0,LoNDon_paris,10045,"[23, 47]",KLM(!),London,Paris
1,MAdrid_miLAN,10055,[],<Air France> (12),Madrid,Milan
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,Budapest_PaRis,10075,[13],12. Air France,Budapest,Paris
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air""",Brussels,London


In [84]:
# add delayCnt column to get delayCnt list length and then max value
df['delayCnt'] = df['RecentDelays'].apply(lambda x : len(x))
maxDelays = max(df['delayCnt'])

# break RecentDelays column into 'delay_1','delay_2','delay_3'

def fillNanValuesTolist(lst,maxVal):
    if len(lst) < maxVal:
        i = 1
        while i <= maxVal - len(lst):
            lst.append(np.nan)
    return lst


df['RecentDelaysClean'] = df['RecentDelays'].apply(lambda x : fillNanValuesTolist(x,maxDelays))


i = 1
while i <= maxDelays:
    df[f'delay_{i}'] = df['RecentDelaysClean'].apply(lambda x : np.nan if np.isnan(x[i-1]) else  x[i-1])
    df[f'delay_{i}']
    i += 1

df

,From_To,FlightNumber,RecentDelays,Airline,from,to,delayCnt,RecentDelaysClean,delay_1,delay_2,delay_3
0,LoNDon_paris,10045,"[23, 47, nan]",KLM(!),London,Paris,2,"[23, 47, nan]",23.0,47.0,NaN
1,MAdrid_miLAN,10055,"[nan, nan, nan]",<Air France> (12),Madrid,Milan,0,"[nan, nan, nan]",NaN,NaN,NaN
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm,3,"[24, 43, 87]",24.0,43.0,87.0
3,Budapest_PaRis,10075,"[13, nan, nan]",12. Air France,Budapest,Paris,1,"[13, nan, nan]",13.0,NaN,NaN
4,Brussels_londOn,10085,"[67, 32, nan]","""Swiss Air""",Brussels,London,2,"[67, 32, nan]",67.0,32.0,NaN


In [85]:
#drop unnecessory columns
df.drop(['RecentDelays','RecentDelaysClean','delayCnt','From_To'], axis = 1,inplace=True)
df

,FlightNumber,Airline,from,to,delay_1,delay_2,delay_3
0,10045,KLM(!),London,Paris,23.0,47.0,NaN
1,10055,<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065,(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels,London,67.0,32.0,NaN


In [86]:
#rearrange the columns appropriately

df = pd.DataFrame(df.loc[:,['FlightNumber','from','to','delay_1','delay_2','delay_3','Airline']])
df

,FlightNumber,from,to,delay_1,delay_2,delay_3,Airline
0,10045,London,Paris,23.0,47.0,NaN,KLM(!)
1,10055,Madrid,Milan,NaN,NaN,NaN,<Air France> (12)
2,10065,London,Stockholm,24.0,43.0,87.0,(British Airways. )
3,10075,Budapest,Paris,13.0,NaN,NaN,12. Air France
4,10085,Brussels,London,67.0,32.0,NaN,"""Swiss Air"""
